In [5]:
import os
import pandas as pd
import asyncio
import nest_asyncio

# ADK Imports
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.genai import types

# --- 0. SETUP ---
nest_asyncio.apply()
# Replace with your actual API Key
os.environ["GOOGLE_API_KEY"] = "your_api_key"

# Retry configuration
retry_config = types.HttpRetryOptions(
    attempts=3, 
    initial_delay=5, 
    http_status_codes=[429, 500, 503]
)

# -------
try:
    df = pd.read_csv('supply_chain.csv')
    print("✅ Historical Data Loaded.")
except:
    print("❌ Error: CSV file not found.")
    df = pd.DataFrame()

# -------

def run_demand_forecast(sku, current_month_factor=1.0):
    if df.empty: return 0
    row = df[df['SKU'] == sku]
    if row.empty: return "Unknown SKU"
    
    # Use .get() to handle missing columns gracefully
    annual_sales = row.iloc[0].get('Number of products sold', 0)
    monthly_forecast = (annual_sales / 12) * current_month_factor
    return int(monthly_forecast)

def run_inventory_check(sku, current_stock_level, seasonality="Normal"):
    factor = 1.2 if seasonality.lower() == "peak" else 1.0
    forecast = run_demand_forecast(sku, factor)
    
    if isinstance(forecast, str): return f"Error: {forecast}"
    
    safety_stock = int(forecast * 0.2)
    target_inventory = forecast + safety_stock
    reorder_qty = target_inventory - current_stock_level
    
    if reorder_qty <= 0:
        return f"Status: OK. (Stock {current_stock_level} > Target {target_inventory})"
    else:
        return f"ACTION: ORDER {reorder_qty} UNITS. (Target {target_inventory} - Stock {current_stock_level})"

def run_logistics_optimization(sku, order_qty):
    if df.empty or order_qty <= 0: return "No shipment needed."
    
    row = df[df['SKU'] == sku]
    if row.empty: return "Unknown SKU"
    
    base = row.iloc[0]
    
    # Normalize to Road Baseline
    current_mode = base.get('Transportation modes', 'Road')
    cost_base = base.get('Shipping costs', 0)
    time_base = base.get('Shipping times', 0) + base.get('Lead times', 0)
    
    if current_mode == 'Air':
        road_cost = cost_base / 3.5
        road_time = time_base / 0.3
    elif current_mode == 'Rail':
        road_cost = cost_base / 0.6
        road_time = time_base - 3
    else:
        road_cost = cost_base
        road_time = time_base

    # Generate Options
    options = [
        {'mode': 'Road', 'cost': road_cost, 'time': road_time},
        {'mode': 'Rail', 'cost': road_cost * 0.6, 'time': road_time + 3},
        {'mode': 'Air', 'cost': road_cost * 3.5, 'time': road_time * 0.3}
    ]
    
    # Selection Logic: Cheapest option
    best_option = min(options, key=lambda x: x['cost'])
    
    return f"Recommended Mode: {best_option['mode']} (${best_option['cost']:.2f}, {best_option['time']:.1f} days)"

# --- THE MASTER COORDINATOR (The AI Manager) ---

coordinator = Agent(
    name="SupplyChainManager",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    instruction="""
    You are the Supply Chain Manager for a real company.
    
    Your goal is to review the operational data provided by the user (Inventory, Forecast, Logistics)
    and make a final executive decision.
    
    YOUR TASK:
    1. Synthesize the inputs into a final executive command.
    2. Explain WHY. (e.g., "Ordering 50 units because stock is low and peak season is coming.")
    3. If no order is needed, just say "Inventory is healthy."
    """,
)

# --- INTERACTIVE EXECUTION ---
async def main():
    print("\n📦 --- SUPPLY CHAIN MANAGEMENT CONSOLE ---")
    
    # --- INTERACTIVE INPUTS ---
    # The code pauses here and waits for user to type the answer
    try:
        target_sku = input("👉 Enter SKU (e.g., SKU0): ").strip()
        
        stock_input = input("👉 Enter Current Stock Level (e.g., 10): ").strip()
        current_stock = int(stock_input)
        
        season_input = input("👉 Enter Season (Normal/Peak): ").strip()
        season = season_input if season_input else "Normal"
        
    except ValueError:
        print("❌ Error: Stock level must be a number.")
        return

    print("-" * 40)
    print(f"🚀 Processing decision for {target_sku}...")
    
    # Run Engines (Python)
    factor = 1.2 if season.lower() == "peak" else 1.0
    forecast_val = run_demand_forecast(target_sku, factor)
    
    if forecast_val == "Unknown SKU":
        print(f"❌ Error: {target_sku} not found in database.")
        return

    inv_decision = run_inventory_check(target_sku, current_stock, season)
    
    try:
        if "ORDER" in inv_decision:
            # Safe extraction of the number
            qty_needed = int(inv_decision.split("ORDER ")[1].split(" ")[0])
            logistics_decision = run_logistics_optimization(target_sku, qty_needed)
        else:
            qty_needed = 0
            logistics_decision = "N/A"
    except:
        qty_needed = 0
        logistics_decision = "Calculation Error"

    print(f"   > Forecast: {forecast_val} units/month")
    print(f"   > Inventory: {inv_decision}")
    print(f"   > Logistics: {logistics_decision}")
    print("-" * 40)
    
    # AI Manager Decision
    runner = InMemoryRunner(agent=coordinator)
    
    prompt = f"""
    Please make a final decision.
    
    DATA CONTEXT:
    - SKU: "{target_sku}"
    - Seasonality: "{season}"
    - Inventory Analysis: "{inv_decision}"
    - Demand Forecast: "{forecast_val}" units/month
    - Logistics Plan: "{logistics_decision}"
    """
    
    try:
        # Await the debug runner to print the output
        await runner.run_debug(prompt)
        print("\n✅ Process Completed.")
        
    except Exception as e:
        print(f"\n❌ API Error: {e}")

# Run it
if __name__ == "__main__":
    await main()

✅ Historical Data Loaded.

📦 --- SUPPLY CHAIN MANAGEMENT CONSOLE ---


👉 Enter SKU (e.g., SKU0):  SKU56
👉 Enter Current Stock Level (e.g., 10):  8
👉 Enter Season (Normal/Peak):  Peak


----------------------------------------
🚀 Processing decision for SKU56...
   > Forecast: 9 units/month
   > Inventory: ACTION: ORDER 2 UNITS. (Target 10 - Stock 8)
   > Logistics: Recommended Mode: Rail ($1.06, 31.0 days)
----------------------------------------

 ### Created new session: debug_session_id

User > 
    Please make a final decision.
    
    DATA CONTEXT:
    - SKU: "SKU56"
    - Seasonality: "Peak"
    - Inventory Analysis: "ACTION: ORDER 2 UNITS. (Target 10 - Stock 8)"
    - Demand Forecast: "9" units/month
    - Logistics Plan: "Recommended Mode: Rail ($1.06, 31.0 days)"
    
SupplyChainManager > Order 2 units of SKU56 via Rail logistics.

**WHY:** The inventory analysis indicates a need to order 2 units to meet the target of 10, bringing current stock from 8 units up to target. With a demand forecast of 9 units/month and the current "Peak" seasonality, maintaining this target inventory level is crucial to avoid stockouts. Utilizing the recommended Rail logistics of